<a href="https://colab.research.google.com/github/yuto90/whisper_gradio/blob/main/gijiroku_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# whisper用
!pip install git+https://github.com/nyanta012/whisper -q
# chatGPT用
!pip install openai

In [ ]:
!pip install gradio==3.4.0

In [ ]:
import gradio as gr 
import whisper

model = whisper.load_model("large")

In [14]:
import openai

# 議事録ファイルを生成
def generate_gijiroku(audio):
    transcription_file = create_transcription_file(audio)
    summaryfile = create_summaryfile(transcription_file)

    return summaryfile

# Whisperで文字起こしファイルを作成
def create_transcription_file(audio):
    results = model.transcribe(audio, verbose=False, language="ja")

    # アウトプット用ファイルを上書きモードで作成して開く
    with open("transcribe.txt", mode="w") as f:
        # 文字起こし結果を書き込み
        for index, _dict in enumerate(results["segments"]):
            f.write(f'{_dict["text"]}\n')

    # 作成したtxtファイルを返却
    return "transcribe.txt"

# chatGPTで要約ファイルを作成
def create_summaryfile(transcription_file):
    # APIキー
    openai.api_key = ""
    
    system_template = """会議の書き起こしが渡されます。
    
    この会議のサマリーをMarkdown形式で作成してください。サマリーは、以下のような形式で書いてください。
    
    - 会議の目的
    - 会議の内容
    - 会議の結果"""
    
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_template},
            {"role": "user", "content": transcription_file}
        ]
    )

    # 回答を取得
    response_text = completion.choices[0].message.content
    
    # アウトプット用ファイルを上書きモードで作成して開く
    with open("summary.txt", mode="w") as f:
        # 結果をアウトプット用ファイルに書き込み
        f.write(f'{response_text}')

    # 作成したtxtファイルを返却
    return "summary.txt"

css = """
        .gradio-container {
            font-family: 'IBM Plex Sans', sans-serif;
        }
        .container {
            max-width: 730px;
            margin: auto;
            padding-top: 1.5rem;
        }
        #gallery {
            min-height: 22rem;
            margin-bottom: 15px;
            margin-left: auto;
            margin-right: auto;
            border-bottom-right-radius: .5rem !important;
            border-bottom-left-radius: .5rem !important;
        }
        #gallery>div>.h-full {
            min-height: 20rem;
        }

"""

block = gr.Blocks(css=css)

with block:   
    gr.Markdown("GIZY")
    with gr.Group():
        with gr.Box():
            with gr.Row(mobile_collapse=False, equal_height=True):
                text_button = gr.Button("generate summary").style(
                        margin=False,
                        rounded=(True, True, True, True),
                    )
        audio = gr.Audio(source="upload", type="filepath", interactive=True)       
        outputs = gr.File(label="summary text file")
        # ボタンを押した時の挙動
        text_button.click(generate_gijiroku, inputs=[audio], outputs=[outputs])

block.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fa75471508a2b903.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://www.huggingface.co/spaces


100%|██████████| 118245/118245 [06:57<00:00, 283.29frames/s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 275, in run_predict
    output = await app.blocks.process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 760, in process_api
    result = await self.call_function(fn_index, inputs, iterator)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 671, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "<ipython-input-14-5f6ab691b1ae>", lin

Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f2efc33e260>,
 'http://127.0.0.1:7861/',
 'https://fa75471508a2b903.gradio.app')